# Hydroserving application tests

In [5]:
!pip install docker
!pip install opencv-python
!apt update && apt install -y libsm6 libxext6 libfontconfig1 libxrender1
!pip install hydro_serving_grpc
!pip install hs==0.0.5

Hit:1 http://security.ubuntu.com/ubuntu xenial-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu xenial InRelease
Hit:3 http://archive.ubuntu.com/ubuntu xenial-updates InReleasem
Hit:4 http://archive.ubuntu.com/ubuntu xenial-backports InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
19 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsm6 is already the newest version (2:1.2.2-1).
libxext6 is already the newest version (2:1.3.3-1).
libxrender1 is already the newest version (1:0.9.9-0ubuntu1).
libfontconfig1 is already the newest version (2.11.94-0ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


!cd /Users/bulat/Documents/Dev/Provectus/hydro-serving && MODEL_DIRECTORY=/Users/bulat/Documents/Dev/Provectus/hydro-serving-runtime/models docker-compose up -d

In [21]:
from hydroserving.httpclient import HydroservingClient
hs_api = HydroservingClient("http://serving-demo.hydrosphere.io")

hs_api.models.list()

[{'model': {'created': '2018-03-06T19:19:20.086',
   'id': 1,
   'modelContract': 'model_name: ""\n',
   'modelType': 'scikit',
   'name': 'iris_svm',
   'source': 'local:iris_svm',
   'updated': '2018-03-06T19:19:20.086'},
  'nextVersion': 1},
 {'lastModelBuild': {'finished': '2018-03-06T19:19:47.821',
   'id': 1,
   'model': {'created': '2018-03-06T19:19:20.201',
    'id': 2,
    'modelContract': 'model_name: "word2vec"\nsignatures {\n  signature_name: "default_spark"\n  inputs {\n    field_name: "text"\n    info {\n      dtype: DT_STRING\n      tensor_shape {\n        dim {\n          size: -1\n          name: ""\n        }\n        unknown_rank: false\n      }\n    }\n  }\n  outputs {\n    field_name: "result"\n    info {\n      dtype: DT_DOUBLE\n      tensor_shape {\n        dim {\n          size: 3\n          name: ""\n        }\n        unknown_rank: false\n      }\n    }\n  }\n}\n',
    'modelType': 'spark:2.1',
    'name': 'word2vec',
    'source': 'local:word2vec',
    'updat

In [9]:
py36 = hs_api.runtimes.create('hydrosphere/serving-runtime-python', '3.6-latest', ['unknown:ssd_demo', "python:3.6"])
print("Done")

HTTPError: HTTP Error 500: Internal Server Error

In [22]:
# build model version
ssd_model = list(filter(lambda m: m['model']['name'] == 'ssd_model_example', hs_api.models.list()))[0]

ssd_model_version = hs_api.models.build(ssd_model['model']['id'])
print(ssd_model_version)

HTTPError: HTTP Error 504: Gateway Timeout

In [25]:
ssd_model_version = hs_api.models.find_version("ssd_model_example", 1)
py36 = hs_api.runtimes.find('hydrosphere/serving-runtime-python', '3.6-latest')
print(py36)
print(ssd_model_version)

{'id': 5, 'name': 'hydrosphere/serving-runtime-python', 'configParams': {}, 'version': '3.6-latest', 'suitableModelType': ['unknown:ssd_demo', 'python:3.6'], 'tags': ['unknown:ssd_demo', 'python:3.6']}
{'id': 3, 'modelVersion': 1, 'imageTag': '1', 'modelName': 'ssd_model_example', 'modelContract': 'model_name: ""\nsignatures {\n  signature_name: "detect"\n  inputs {\n    field_name: "image_b64"\n    info {\n      dtype: DT_STRING\n    }\n  }\n  outputs {\n    field_name: "bboxes"\n    info {\n      dtype: DT_DOUBLE\n      tensor_shape {\n        dim {\n          size: -1\n          name: ""\n        }\n        dim {\n          size: 4\n          name: ""\n        }\n        unknown_rank: false\n      }\n    }\n  }\n  outputs {\n    field_name: "scores"\n    info {\n      dtype: DT_DOUBLE\n      tensor_shape {\n        dim {\n          size: -1\n          name: ""\n        }\n        unknown_rank: false\n      }\n    }\n  }\n  outputs {\n    field_name: "classes"\n    info {\n      dtyp

In [27]:
# create application
application = {
  "name": "TEST_APP",
  "executionGraph": {
    "stages": [
      {
        "services": [
          {
            "runtimeId": py36["id"],
            "modelVersionId": ssd_model_version["id"],
            "weight": 100,
            "signatureName": "detect"
          }
        ]
      }
    ]
  },
  "kafkaStreaming": []
}

created_app = hs_api.applications.create(application)
print(created_app)

{'kafkaStreaming': [], 'id': 2, 'name': 'TEST_APP', 'contract': 'model_name: ""\nsignatures {\n  signature_name: "detect"\n  inputs {\n    field_name: "image_b64"\n    info {\n      dtype: DT_STRING\n    }\n  }\n  outputs {\n    field_name: "bboxes"\n    info {\n      dtype: DT_DOUBLE\n      tensor_shape {\n        dim {\n          size: -1\n          name: ""\n        }\n        dim {\n          size: 4\n          name: ""\n        }\n        unknown_rank: false\n      }\n    }\n  }\n  outputs {\n    field_name: "scores"\n    info {\n      dtype: DT_DOUBLE\n      tensor_shape {\n        dim {\n          size: -1\n          name: ""\n        }\n        unknown_rank: false\n      }\n    }\n  }\n  outputs {\n    field_name: "classes"\n    info {\n      dtype: DT_STRING\n      tensor_shape {\n        dim {\n          size: -1\n          name: ""\n        }\n        unknown_rank: false\n      }\n    }\n  }\n}\n', 'executionGraph': {'stages': [{'services': [{'serviceDescription': {'runtimeI

In [28]:
%matplotlib inline
from show_result import show_result
import hydro_serving_grpc as hs
import os
import grpc
import base64

def serve_folder(pictures_folder):
    channel = grpc.insecure_channel('sidecar:8080')
    client = hs.PredictionServiceStub(channel=channel)

    pics_dir = pictures_folder
    for file in os.listdir(pics_dir):
        full_file_path = os.path.join(pics_dir, file)

        if (not os.path.isfile(full_file_path)):
            print("Skipped: {}".format(full_file_path))
            continue

        print("Serving: {}".format(full_file_path))

        with open(full_file_path, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read())

        image_tensor = hs.TensorProto(
            dtype=hs.DT_STRING,
            string_val=[encoded_string]
        )

        request = hs.PredictRequest(
            model_spec=hs.ModelSpec(signature_name="detect", name="TEST_APP"),
            inputs={
                "image_b64": image_tensor,
            }
        )

        result = client.Predict(request)

        show_result(full_file_path, result)

In [8]:
serve_folder("./pics")

NameError: name 'grpc' is not defined

In [7]:
serve_folder("./cars")

NameError: name 'grpc' is not defined

In [6]:
serve_folder("./buldozers")

NameError: name 'grpc' is not defined